# Elasticsearch + NumPy + Pandas

https://kb.objectrocket.com/elasticsearch/how-to-use-elasticsearch-data-using-pandas-in-python-349
https://elasticsearch-py.readthedocs.io/en/v7.12.0/

In [25]:
import json
import os
from os import listdir
from os.path import join
import re
import requests
from elasticsearch import Elasticsearch

from tqdm.notebook import tqdm

tweetsPath = '../data/tweets'
speechesPath = '../data/speeches'

tweetsIndex = 'tfg-tweet'
speechesIndex = 'tfg-speech'

# Define config
host = "my-elasticsearch"
username= 'elastic'
password= 'password'
port = 9200
timeout = 1000

# Init Elasticsearch instance
es = Elasticsearch(
    [{'host': host, 'port': port}],
    timeout=timeout,
    http_auth=(username, password)
)
es.info()


{'name': 'my-elasticsearch',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'Z4gQIZiYRui8ZBubV3zK3A',
 'version': {'number': '7.10.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa',
  'build_date': '2020-12-05T01:00:33.671820Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [22]:
tweetsPaths = [join(tweetsPath, f) for f in listdir(tweetsPath)]
speechesPaths = [join(speechesPath, f) for f in listdir(speechesPath)]

files = tweetsPaths + speechesPaths

In [24]:
with open(files[0], 'r') as f:
    text = f.read()
    data = json.loads(text)
    
    res = es.index(index="test-index",  body=data)
    print(res)

{'_index': 'test-index', '_type': '_doc', '_id': 'apGJt3gBG0gO2PiuRllt', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}


In [27]:
for file in tqdm(tweetsPaths):
    with open(file, 'r') as f:
        text = f.read()
        data = json.loads(text)
        es.index(index=tweetsIndex, body=data)

In [33]:
for file in tqdm(speechesPaths):
    with open(file, 'r') as f:
        text = f.read()
        data = json.loads(text)
        speech = {}
        speech['text'] = data['text']
        speech['freeling'] = data['freeling']
        speech['created_at'] = data['metadata']['celebrated']
        speech['debate'] = data['metadata']['debate']
        speech['diputado'] = data['metadata']['diputado'][0] 
        try:
            es.index(index=speechesIndex, body=speech)
        except:
            print(file)

  0%|          | 0/17218 [00:00<?, ?it/s]

../data/speeches/17217-DSCD-14-PL-002.json


{
    "metadata": {
        "celebrated": "2021-05-25T00:00:00.000Z",
        "debate": "CONVALIDACI\u00d3N O DEROGACI\u00d3N DE REALES DECRETOS-LEYES: -- REAL DECRETO-LEY 4/2021, DE 9 DE MARZO, POR EL QUE SE MODIFICAN LA LEY 27/2014, DE 27 DE NOVIEMBRE, DEL IMPUESTO SOBRE SOCIEDADES, Y EL TEXTO REFUNDIDO DE LA LEY DEL IMPUESTO SOBRE LA RENTA DE NO RESIDENTES, APROBADO MEDIANTE REAL DECRETO LEGISLATIVO 5/2004, DE 5 DE MARZO, EN RELACI\u00d3N CON LAS ASIMETR\u00cdAS H\u00cdBRIDAS. ",
        "diputado": [
            {
                "apellidos": "Batet Lama\u00f1a",
                "apellidosNombre": "Batet Lama\u00f1a, Meritxell",
                "codParlamentario": 215,
                "fchAlta": "26/11/2019",
                "fchBaja": "",
                "formacion": "PSC-PSOE",
                "genero": 2,
                "grupo": "Grupo Parlamentario Socialista",
                "idCircunscripcion": 8,
                "idLegislatura": 14,
                "nombre": "Meritxell",
 